In [1]:
import pandas as pd 
import numpy as np
import os
import re
import warnings
warnings.filterwarnings("ignore")
pwd=os.getcwd()
dataset = pd.read_csv(pwd + r"//functional_difficulty.csv", encoding="ISO-8859-1")
#region_total_population=pd.read_csv("region_total_population.csv")


In [2]:
dataset.dropna().drop_duplicates()
dataset=dataset.drop(dataset[~dataset["Status"].isin(['All', 'Mild', 'Moderate', 'Severe'])].index)

In [3]:
dataset.info()
dataset.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 118292 entries, 0 to 118296
Data columns (total 29 columns):
 #   Column                                                                Non-Null Count   Dtype 
---  ------                                                                --------------   ----- 
 0   Region                                                                118292 non-null  object
 1   Province                                                              118292 non-null  object
 2   Mun                                                                   118292 non-null  object
 3   RegCode_New                                                           118292 non-null  object
 4   RegCode_Old                                                           118292 non-null  object
 5   ProvCode_New                                                          118292 non-null  object
 6   ProvCode_Old                                                          117716 non-null  object
 7 

,Age 5-9,Age 10-14,Age 15 - 19,Age 20 - 24,Age 25 - 29,Age 30 - 34,Age 35 - 39,Age 40 - 44,Age 45 - 49,Age 50 - 54,Age 55 - 59,Age 60 - 64,Age 65 - 69,Age 70 - 74,Age 75 - 79,Age 80 years and over
count,118292.000000,118292.00000,118292.000000,118292.000000,118292.000000,118292.000000,118292.000000,118292.000000,118292.000000,118292.000000,118292.000000,118292.000000,118292.000000,118292.000000,118292.000000,118292.000000
mean,7.263044,7.42434,8.683664,10.451966,11.257363,11.920544,13.559801,23.332533,32.820749,41.972069,45.425861,51.945829,48.636155,45.023535,35.799065,57.706827
std,26.809068,34.41203,54.772227,72.883309,75.869223,72.019236,76.125514,136.689187,195.608401,241.392551,242.343841,249.503966,204.218072,166.669915,115.979131,160.509864
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,6.000000
50%,2.000000,2.00000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,3.000000,4.000000,5.000000,7.000000,7.000000,9.000000,9.000000,19.000000
75%,6.000000,6.00000,6.000000,7.000000,7.000000,8.000000,9.000000,12.000000,15.000000,21.000000,25.000000,34.000000,36.000000,38.000000,34.000000,59.000000
max,1814.000000,3089.00000,5248.000000,6838.000000,7245.000000,6517.000000,6357.000000,10443.000000,16164.000000,19872.000000,20334.000000,20893.000000,17443.000000,14032.000000,10714.000000,11525.000000


In [4]:
dataset["Status"].unique()

array(['All', 'Mild', 'Moderate', 'Severe'], dtype=object)

In [5]:
code_to_keep = [x for x in dataset.columns if not re.match(r"^.*(Mun|MunCode_New|Old)$", x)]
#combine agekeep with this using and
dataset=dataset[code_to_keep]

In [6]:
age={"Age: 5-19":["Age 5-9","Age 10-14","Age 15 - 19"],
                        "Age: 20-34":["Age 20 - 24","Age 25 - 29","Age 30 - 34"],
                        "Age: 35–54":["Age 35 - 39","Age 40 - 44","Age 45 - 49","Age 50 - 54"],
                        "Age: 55-80+":["Age 55 - 59","Age 60 - 64","Age 65 - 69","Age 70 - 74","Age 75 - 79",'Age 80 years and over']
                        }
for col_age,age_value in age.items():
    existing=[col for col in age_value if col in dataset.columns]
    dataset[col_age]=dataset[age_value].sum(axis=1)

    
#try combine this in for loop for more effiencent 
#dont every use gpt next time even unless its taking you 1 hour and your stuck or you got a error you never got before but you are allowed to
#search but its it not an error just keep tryingif you think you can do it and you just need more time to think then wiat for 1 hours  before 
#gpt in those time think hard on how to solve the problem this is for your future (improve your creativity and problem solving skill)

In [7]:

age_to_keep = [x for x in dataset.columns if not re.match(r"^(\bAge \b).*", x)]
dataset=dataset[age_to_keep]
age_to_keep


['Region',
 'Province',
 'RegCode_New',
 'ProvCode_New',
 'Disability',
 'Sex',
 'Status',
 'Household Population 5 Years Old and Over with Functional Difficulty',
 'Age: 5-19',
 'Age: 20-34',
 'Age: 35–54',
 'Age: 55-80+']

In [8]:
dataset["Disability"]=dataset["Disability"].str.strip()
rename={"Visual Disability":'Seeing even if wearing eyeglasses',
                        "Hearing Disability":'Hearing even if using a hearing aid',
                        "Mobility Disability":'Walking or climbing steps',
                        "Cognitive Disability":'Remembering or concentrating',
                        "Self-Care Disability":'Self-caring (washing all over or dressing)',
                        "Communication Disability":"Communicating"}
for Disability_new,Disability_old in rename.items():
   check=[col for col in Disability_old if col in dataset]
   dataset["Disability"].replace({Disability_old:Disability_new},inplace=True)


In [9]:
to_category=["Region",
                        "Province",
                        "Disability",
                        "Sex",
                        "Status"
                      ]
for x in to_category:
    to_unique=[region_code for region_code in dataset[x].unique()]
    dataset[x]=pd.CategoricalIndex(dataset[x], categories=to_unique)

In [10]:
to_int64=[x for x in dataset.columns]

for x in to_int64:
    dataset[x]=dataset[x].convert_dtypes()



In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118292 entries, 0 to 118296
Data columns (total 12 columns):
 #   Column                                                                Non-Null Count   Dtype   
---  ------                                                                --------------   -----   
 0   Region                                                                118292 non-null  category
 1   Province                                                              118292 non-null  category
 2   RegCode_New                                                           118292 non-null  string  
 3   ProvCode_New                                                          118292 non-null  string  
 4   Disability                                                            118292 non-null  category
 5   Sex                                                                   118292 non-null  category
 6   Status                                                                118292 non-

In [12]:
to_numeric = [x for x in dataset.columns if re.match(r"^(\bAge|Ho).*", x)]
to_numeric
for x in to_numeric:
    dataset[x]=pd.to_numeric(dataset[x],errors="coerce").fillna(0).astype("int64")

In [23]:
dataset.loc[117937] #just dont show that you drop this column

Region                                                                  BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANA...
Province                                                                                                 INTERIM PROVINCE
RegCode_New                                                                                                  PH1900000000
ProvCode_New                                                                                                 PH1999900000
Disability                                                                                              Visual Disability
Sex                                                                                                            Both Sexes
Status                                                                                                                All
Household Population 5 Years Old and Over with Functional Difficulty                                                  304
Age: 5-19               